In [1]:
import Pkg;
Pkg.activate(joinpath(@__DIR__, ".."));
Pkg.instantiate();
using DelimitedFiles
using CSV
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random;

  Activating environment at `~/planar_quadruped_landing_real_quadruped/Project.toml`


In [2]:
include("quadratic_cost.jl")
include("real_quadruped.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl");

WebIO._IJuliaInit()

In [3]:
h = 1
v_init_z = sqrt(2 * 9.81 * h)

4.4294469180700204

In [4]:
# Dynamics model
model = RealQuadruped()

# Some parameters
dt = 0.02
N = 41
times = range(0, dt * (N - 1), length=N)
k_trans = 15
init_mode = 1
n = 37
m = 13

# initial state
xinit = zeros(n)

xinit[3] = 0.5  # zb

xinit[5] = deg2rad(10) # pitch

xinit[7] = 0.15 # x1
xinit[8] = 0.1  # y1

xinit[10] = -0.15 # x2
xinit[11] = 0.1   # y2
xinit[12] = 0.1   # z2

xinit[13] = 0.15 # x3
xinit[14] = -0.1 # y3

xinit[16] = -0.15 # x4
xinit[17] = -0.1  # y4
xinit[18] = 0.1   # z4

xinit[21] = -v_init_z # vzb

# Desired final state
xterm = zeros(n)

xterm[3] = 0.3 # zb

xterm[7] = 0.15 # x1
xterm[8] = 0.1  # y1

xterm[10] = -0.15 # x2
xterm[11] = 0.1   # y2

xterm[13] = 0.15 # x3
xterm[14] = -0.1 # y3

xterm[16] = -0.15 # x4
xterm[17] = -0.1  # y4

-0.1

In [5]:
# Reference Trajectory
Xref, Uref = reference_trajectory(model, N, k_trans, xterm, dt);

In [6]:
# Objective
Q_diag = 10.0 * ones(n)
Q_diag[end] = 0.0
Q = Diagonal(Q_diag)

R_diag = 1e-3 * ones(m)
R_diag[end] = 0.0
R = Diagonal(R_diag)

Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end

push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]));

In [7]:
# Define the NLP
nlp = HybridNLP(model, obj, init_mode, k_trans, N, xinit, xterm);

In [8]:
# initialize guess
Xguess = [zeros(n) for x in Xref]
k_trans = nlp.k_trans

for k = 1:N
    if k <= k_trans
        Xguess[k] = xinit + (xterm - xinit) / (k_trans - 1) * (k - 1)
    else
        Xguess[k][1:36] = xterm[1:36]
    end
end

for k = 1:N-1
    if k < k_trans
        Xguess[k+1][end] = Xguess[k][end] + 0.018
    else
        Xguess[k+1][end] = Xguess[k][end] + 0.02
    end
end

mb, g = model.mb, model.g
Uguess = [zeros(m) for u in Uref]

for k = 1:N-1
    if k < k_trans
        if init_mode == 1
            Uguess[k][3] = -mb * g / 2
            Uguess[k][9] = -mb * g / 2
        else
            Uguess[k][6] = -mb * g / 2
            Uguess[k][12] = -mb * g / 2
        end
        Uguess[k][end] = 0.018
    else
        Uguess[k][3] = -mb * g / 4
        Uguess[k][6] = -mb * g / 4
        Uguess[k][9] = -mb * g / 4
        Uguess[k][12] = -mb * g / 4
        Uguess[k][end] = dt
    end
end

Z0 = packZ(nlp, Xguess, Uguess);

In [9]:
Z_sol, solver = solve(Z0, nlp, c_tol=1e-3, tol=1e-3)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:  3440493
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     2037
                     variables with only lower bounds:      160
                variables with lower and upper bounds:    

([0.013290773425182275, -1.5544517306287e-5, 0.5000900091708156, 2.6191479178706805e-7, 0.1745334263269142, 4.4946728455851077e-7, 0.14994979279278567, 0.1000022780135694, -3.3105918957400028e-6, -0.14999831742853942  …  -9.256e-320, 0.0, -1.1965e-319, 0.0, 1.07133e-319, 0.0, 1.01906e-319, 0.0, -6.519e-320, 0.7677864282583309], Ipopt.Optimizer)

In [10]:
data_idx = 3

if isdir("../results/data_" * string(data_idx)) == false
    mkdir("../results/data_" * string(data_idx))
end

writedlm("../results/data_" * string(data_idx) * "/data_" * string(data_idx) * ".csv", Z_sol, ',')